In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import LSTM 
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LeakyReLU

# Function to perform pre-processing steps

At first tokenization will be performed then stemming will be performed over tokens

In [91]:
def token_stems(text):
        
    tokens=tokenizing(text) 
    stems=stemming(tokens)
    return stems       

# Stemming Function

In [92]:
from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")


def stemming(text):
    
    stems =[stemmer.stem(t) for t in text]
    return stems

# Tokenization Function

In [93]:
def tokenizing(text):
    
    #breaking each word and making them tokens
    tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #storing only alpha tokens
    filtered_tokens=[]
    for token in tokens:
        if (re.search('[a-zA-Z]|\'', token)):
            filtered_tokens.append(token)

    return filtered_tokens

In [94]:
train= pd.read_excel("D:\\virtual\\dataset.xlsx")
train= train[train['Type']== 'Leave_Approval']
train.shape

(148, 7)

In [95]:
docs= train['Data']

In [96]:
train.head()

,S.no,Data,Module,Type,sub_type,sub_type_two,sub_type_three
1279,1280,approve person leave on my behalf,Leave,Leave_Approval,particular,approve,NaN
1280,1281,approve person leave,Leave,Leave_Approval,particular,approve,NaN
1281,1282,approve person leave request,Leave,Leave_Approval,particular,approve,NaN
1282,1283,approve person leaves,Leave,Leave_Approval,particular,approve,NaN
1283,1284,kindly approve person leave request,Leave,Leave_Approval,particular,approve,NaN


In [97]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)


In [98]:
#print(docs[1],"\n\n",tokens[1])

In [99]:
x, y = np.asarray(tokens) , np.asarray(train['sub_type'])

In [100]:
le = LabelEncoder()
y = le.fit_transform(y)
y= to_categorical(y)
#y[:10]

In [101]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [102]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [103]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [104]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
#test_sequences_matrix[1]

# Model 

In [105]:
def model():
    
    #Create model by simply calling the Sequential constructor
    
    model = Sequential()
    
    #This layer can only be used as the first layer in a model.
    
    model.add(Embedding(max_words,50,input_length=max_len))
    
    #specify number of neurons in lstm layer and the activation function that we want to use.
    
    model.add(LSTM(64, activation='tanh',dropout=0.01))
    
    #specify number of neurons in dense layer and the activation function that we want to use.
    
    #model.add(Dense(50, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.01))
    
    #and we create our output layer with two nodes as we have 2 class labels.
    
    model.add(Dense(2, activation='softmax'))
    
    #Now for training, we need to define an optimizer, loss measure and the error metric.
    # We will use the binary_crossentropy as our loss measure. 
    #As for the minimization algorithm, we will use "rmsprop".
    #This optimizer is usually a good choice for recurrent neural networks.
    
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
    return model

In [110]:
c= model()

c.fit(sequences_matrix,y,epochs=15, validation_split=0.3)

Train on 103 samples, validate on 45 samples
Epoch 1/15
103/103 [==============================] - 2s 23ms/step - loss: 0.6877 - acc: 0.5146 - val_loss: 0.7007 - val_acc: 0.2667
Epoch 2/15
103/103 [==============================] - 1s 5ms/step - loss: 0.6645 - acc: 0.5631 - val_loss: 0.6750 - val_acc: 0.7111
Epoch 3/15
103/103 [==============================] - 1s 5ms/step - loss: 0.6416 - acc: 0.7087 - val_loss: 0.6962 - val_acc: 0.2889
Epoch 4/15
103/103 [==============================] - 1s 5ms/step - loss: 0.6044 - acc: 0.5728 - val_loss: 0.6359 - val_acc: 0.8000
Epoch 5/15
103/103 [==============================] - 1s 6ms/step - loss: 0.5604 - acc: 0.7670 - val_loss: 0.5729 - val_acc: 0.8667
Epoch 6/15
103/103 [==============================] - 1s 5ms/step - loss: 0.5090 - acc: 0.8641 - val_loss: 0.5169 - val_acc: 0.8889
Epoch 7/15
103/103 [==============================] - 1s 5ms/step - loss: 0.4968 - acc: 0.8058 - val_loss: 0.4660 - val_acc: 0.9556
Epoch 8/15
103/103 [==========

In [111]:
"""from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit


#m = model()
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=model, epochs=5, verbose=2)
kfold = ShuffleSplit(n_splits=10, test_size=0.3, random_state=100)
m.fit(cross_val_score(estimator,sequences_matrix,y, cv=kfold))"""


'from keras.layers import Dense\nfrom keras.models import Sequential\nfrom keras.wrappers.scikit_learn import KerasClassifier\nfrom sklearn.model_selection import StratifiedKFold\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import ShuffleSplit\n\n\n#m = model()\n# evaluate model with standardized dataset\nestimator = KerasClassifier(build_fn=model, epochs=5, verbose=2)\nkfold = ShuffleSplit(n_splits=10, test_size=0.3, random_state=100)\nm.fit(cross_val_score(estimator,sequences_matrix,y, cv=kfold))'

In [112]:
accuracy = c.evaluate(test_sequences_matrix,ytest)

45/45 [==============================] - 0s 1ms/step


In [113]:
#save your model
c.save('Type_Leave.h5')

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import LSTM 
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LeakyReLU

Using TensorFlow backend.


In [2]:
def token_stems(text):
    tokens=tokenizing(text) 
    stems=stemming(tokens)
    return stems       

In [3]:
from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")


def stemming(text):
    
    stems =[stemmer.stem(t) for t in text]
    return stems

In [4]:
def tokenizing(text):
    
    #breaking each word and making them tokens
    tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #storing only alpha tokens
    filtered_tokens=[]
    for token in tokens:
        if (re.search('[a-zA-Z]|\'', token)):
            #if token not in stopwords:
            filtered_tokens.append(token)

    return filtered_tokens

In [5]:
train= pd.read_excel("D:\\virtual\\leave_final.xlsx")

In [6]:
train = train[train['sub_type_two'] == 'specific']

In [7]:
docs= train['Data']

In [8]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)


In [9]:
x, y = np.asarray(tokens) , np.asarray(train['TypeLeave'])

In [10]:
le = LabelEncoder()
y = le.fit_transform(y)
y= to_categorical(y)

In [11]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [12]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [13]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [14]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [22]:
def model():
    
    #Create model by simply calling the Sequential constructor
    
    model = Sequential()
    
    #This layer can only be used as the first layer in a model.
    
    model.add(Embedding(max_words,50,input_length=max_len))
    
    model.add(LSTM(128, input_shape=(xtrain.shape[1:]), activation='tanh', return_sequences=True))
    
    model.add(Dropout(0.2))
    
    #specify number of neurons in lstm layer and the activation function that we want to use.
    
    model.add(LSTM(64, activation='tanh',dropout=0.01))
    
    #specify number of neurons in dense layer and the activation function that we want to use.
    
    model.add(Dense(50, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.01))
    
    #and we create our output layer with two nodes as we have 2 class labels.
    
    model.add(Dense(25, activation='relu'))
    
    model.add(Dropout(0.01))
    
    model.add(Dense(10, activation='relu'))
    
    model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    
    #Now for training, we need to define an optimizer, loss measure and the error metric.
    # We will use the binary_crossentropy as our loss measure. 
    #As for the minimization algorithm, we will use "rmsprop".
    #This optimizer is usually a good choice for recurrent neural networks.
    
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
    return model

In [23]:
c= model()

c.fit(sequences_matrix,y,epochs=30, validation_split=0.1)

Train on 529 samples, validate on 59 samples
Epoch 1/30
529/529 [==============================] - 7s 14ms/step - loss: 0.6394 - acc: 0.6125 - val_loss: 0.9413 - val_acc: 0.0678
Epoch 2/30
529/529 [==============================] - 5s 10ms/step - loss: 0.4192 - acc: 0.8866 - val_loss: 0.8663 - val_acc: 0.4576
Epoch 3/30
529/529 [==============================] - 5s 10ms/step - loss: 0.2589 - acc: 0.9263 - val_loss: 0.9486 - val_acc: 0.4746
Epoch 4/30
529/529 [==============================] - 6s 11ms/step - loss: 0.1083 - acc: 0.9716 - val_loss: 1.3649 - val_acc: 0.3898
Epoch 5/30
529/529 [==============================] - 6s 11ms/step - loss: 0.0868 - acc: 0.9679 - val_loss: 0.5542 - val_acc: 0.8475
Epoch 6/30
529/529 [==============================] - 6s 10ms/step - loss: 0.0671 - acc: 0.9811 - val_loss: 0.2475 - val_acc: 0.9153
Epoch 7/30
529/529 [==============================] - 5s 10ms/step - loss: 0.1227 - acc: 0.9603 - val_loss: 0.3931 - val_acc: 0.8983
Epoch 8/30
529/529 [====

In [24]:
c.save('Leave_inquiry_specific_Model.h5')

# QUERY LEAVE SPECIFIC

In [25]:
train= pd.read_excel("D:\\virtual\\leave_final.xlsx")

In [26]:
train = train[train['sub_type_two'] == 'specific']

In [27]:
from keras.models import load_model
m = load_model('Leave_inquiry_specific_Model.h5')
max_len = 300

In [28]:
docs= train['Data']

In [29]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)

In [30]:
x, y = np.asarray(tokens) , np.asarray(train['TypeLeave'])

In [31]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [32]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [33]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [34]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [60]:
query = 'how many sick leaves i have taken'
query = 'how many leaves i have availed'

In [55]:
query = 'how many sick leaves have left'
query = 'how many sick leaves i can take more'
query = 'Can you please inform me about how many sick leaves can i take more'

In [61]:
sent = query

In [62]:
sen = token_stems(sent)
sen

['how', 'mani', 'leav', 'i', 'have', 'avail']

In [63]:
sen_test = ([list(sen)])
sen_sequences = tok.texts_to_sequences(sen_test)
sen_sequences_matrix = sequence.pad_sequences(sen_sequences,maxlen=max_len)

In [64]:
score = m.predict(sen_sequences_matrix)
print(score)
print(max(score[0]))

[[1.0000000e+00 3.3748933e-08]]
1.0
